# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [81]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [83]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,293.32,79,40,5.66,US,1719836502
1,1,makurazaki,31.2667,130.3167,298.34,92,100,3.65,JP,1719836503
2,2,iqaluit,63.7506,-68.5145,281.00,66,75,6.69,CA,1719836407
3,3,san luis de la loma,17.2706,-100.8939,298.59,90,100,2.41,MX,1719836506
4,4,bredasdorp,-34.5322,20.0403,291.58,58,0,2.58,ZA,1719836509


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [85]:
%%capture --no-display

# Configure the map plot
map_plot= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City")    

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [87]:
# Narrow down cities that fit criteria and drop any results with null values  
# My ideal weather: Wind speed less than 5 m/s; Humidity less than 75
Ideal_cities = city_data_df.loc[(city_data_df["Wind Speed"]<4)&(city_data_df["Wind Speed"]>3)\
                                & (city_data_df["Humidity"]<50)&(city_data_df["Cloudiness"]<30)]

# Drop any rows with null values
Ideal_cities_clean = Ideal_cities.dropna()

# Display sample data
Ideal_cities_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,vorgashor,67.5833,63.9500,285.19,42,12,3.67,RU,1719836586
87,87,antsiranana,-12.2787,49.2917,303.57,45,20,3.09,MG,1719836655
221,221,hotan,37.0997,79.9269,307.64,15,4,3.32,CN,1719836876
334,334,priob'ye,62.5206,65.5964,293.04,32,9,3.45,RU,1719837097
394,394,al jawf,29.5000,38.7500,310.54,12,0,3.61,SA,1719837213
441,441,oblivskaya,48.5362,42.5014,306.02,23,4,3.14,RU,1719837299
453,453,mgandu,-5.9500,34.1333,299.56,25,11,3.19,TZ,1719837317


### Step 3: Create a new DataFrame called `hotel_df`.

In [89]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Ideal_cities_clean[["City","Country","Lat","Lng","Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

/var/folders/d7/tvjrqth10gb5pst1dd7n9wl40000gn/T/ipykernel_89684/1766342809.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
52,vorgashor,RU,67.5833,63.9500,42,
87,antsiranana,MG,-12.2787,49.2917,45,
221,hotan,CN,37.0997,79.9269,15,
334,priob'ye,RU,62.5206,65.5964,32,
394,al jawf,SA,29.5000,38.7500,12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [97]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
limit = 1

params = {"categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url,params = params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vorgashor - nearest hotel: София
antsiranana - nearest hotel: Central
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
priob'ye - nearest hotel: Жемчужина
al jawf - nearest hotel: No hotel found
oblivskaya - nearest hotel: No hotel found
mgandu - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
52,vorgashor,RU,67.5833,63.9500,42,София
87,antsiranana,MG,-12.2787,49.2917,45,Central
221,hotan,CN,37.0997,79.9269,15,Hua Yi International Hotel (accepts foreigners)
334,priob'ye,RU,62.5206,65.5964,32,Жемчужина
394,al jawf,SA,29.5000,38.7500,12,No hotel found
441,oblivskaya,RU,48.5362,42.5014,23,No hotel found
453,mgandu,TZ,-5.9500,34.1333,25,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [119]:
%%capture --no-display

# Configure the map plot
map_plot_ideal= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols=["Hotel Name","City","Country","Lat","Lng","Humidity"])   

# Display the map
map_plot_ideal

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)